In [2]:
!ls

sample_data


In [3]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/My Drive/Urban Systems Studio/Roof-Type-Classification-master")


Mounted at /content/drive


In [4]:
!pip install opencv-python
!pip install numpy
!pip install geopandas

In [5]:
from tkinter.filedialog import askopenfilename as ask



In [6]:
import os
os.environ['OPENCV_IO_MAX_IMAGE_PIXELS']=str(2**64)


In [7]:
import cv2


In [8]:
import numpy as np
import json
import geopandas as gpd

In [35]:
class PrepareData():
    def __init__(self):
        self.executeMode = input("Declare mode of execution (TestData/TrainData): ")
        with open('Atlanta_03.tfw', 'rb') as file:  # Open in binary mode
            self.worldRaw = file.read()

        # Read the image file in binary mode without attempting to decode it
        with open('Atlanta_03.jpg', 'rb') as file:
            self.imageData = file.read()  # Read image data directly

        self.ImageInContext = cv2.imdecode(np.frombuffer(self.imageData, np.uint8), cv2.IMREAD_COLOR)

        # Specify the correct path to the shapefile
        shapefile_path = 'Atlanta_03.shp'
        self.buildShape = gpd.read_file(shapefile_path)  # Read shapefile using the correct path
        self.worldInformation()
        self.handleImage()
        self.createWorldFiles()

    # Remaining code...


    def worldInformation(self):
      # Decode the bytes-like object to a string
      decoded_world_raw = self.worldRaw.decode('utf-8')
      # Split the decoded string by newline character
      world_raw_lines = decoded_world_raw.split("\n")
      # Check for and remove empty lines
      world_raw_lines = [line for line in world_raw_lines if line]
      # Extract the required values
      self.X = float(world_raw_lines[4])
      self.Y = float(world_raw_lines[5])
      self.Xsize = float(world_raw_lines[0])
      self.Ysize = float(world_raw_lines[3])
      self.worldInfo = [self.X, self.Y, self.Xsize, self.Ysize]


    def handleImage(self):
        self.boundingBoxes = [[], [], [], []]
        print("Building Borders Calculating...")

        # Check for and fix invalid geometries
        self.buildShape.geometry = self.buildShape.geometry.buffer(0)
        self.buildShape = self.buildShape[self.buildShape.geometry.is_valid]

        for shp in range(len(self.buildShape)):
            self.boundingBoxes[0].append(int((self.buildShape.envelope[shp].bounds[0]-self.X)/self.Xsize))
            self.boundingBoxes[1].append(int((self.buildShape.envelope[shp].bounds[1]-self.Y)/self.Ysize))
            self.boundingBoxes[2].append(int((self.buildShape.envelope[shp].bounds[2]-self.X)/self.Xsize))
            self.boundingBoxes[3].append(int((self.buildShape.envelope[shp].bounds[3]-self.Y)/self.Ysize))

        for i in range(len(self.buildShape)):
            cv2.imwrite("{}/{}_{}.jpg".format(self.executeMode, self.buildShape.Polygon_ID[i], self.buildShape.Image_Name[0]),
                        self.ImageInContext[self.boundingBoxes[3][i]:self.boundingBoxes[1][i], self.boundingBoxes[0][i]:self.boundingBoxes[2][i]])
        print("Calculated!")

    def createWorldFiles(self):
        for k in range(len(self.buildShape)):
            with open('{}/{}_{}.txt'.format(self.executeMode, self.buildShape.Polygon_ID[k], self.buildShape.Image_Name[0]), 'w') as f:
                json.dump({"Upper Left X":self.buildShape.envelope[k].bounds[0], "Upper Left Y: ":self.buildShape.envelope[k].bounds[1]}, f)

A = PrepareData()


Declare mode of execution (TestData/TrainData): TestData
Building Borders Calculating...


error: OpenCV(4.8.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:787: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'
